In [2]:
import numpy as np 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from plain_cnn_ensemble import ensemble_cnn

In [2]:
dataflow = ImageDataGenerator(rescale=1./255)

In [3]:
flow = dataflow.flow_from_directory('data',
                                    target_size=(32, 32),
                                    batch_size=256,
                                    shuffle=False,
                                    class_mode='input')

Found 300000 images belonging to 1 classes.


In [4]:
def predict_flow(X):
    y_sum = np.zeros((300000, 10))


    for model_number in range(1, 4):
        for iteration in range(1, 11):
            model_name = f'model_{model_number}_{iteration}'

            model = tf.keras.models.load_model(f'./models/with_aug/{model_name}.h5')

            y_proba = model.predict(X)
            y_hat = np.zeros_like(y_proba)
            y_hat[np.arange(y_proba.shape[0]), np.argmax(y_proba, axis=1)] = 1

            y_sum = y_sum + y_hat

            model=None


    y_sum = y_sum + np.random.uniform(-0.1, 0.1, y_sum.shape)
    y_hat_final = np.zeros_like(y_sum)
    y_hat_final[np.arange(y_sum.shape[0]), np.argmax(y_sum, axis=1)] = 1

    return y_hat_final

In [5]:
y_hat = predict_flow(flow)

1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10


In [6]:
y_hat_simple = np.argmax(y_hat, axis=1)

In [7]:
y_hat_simple

array([9, 3, 2, ..., 3, 5, 6])

In [8]:
resultDF = pd.DataFrame()

In [9]:
resultDF['pred'] = y_hat_simple

In [10]:
resultDF['id'] = list(range(1, 300001))

In [11]:
resultDF['label'] = None

In [12]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [13]:
for i in range(10):
    resultDF['label'][resultDF['pred'] == i] = label_names[i]

/home/krzysztof/.conda/envs/ml-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
resultDF[['id', 'label']]

,id,label
0,1,truck
1,2,cat
2,3,bird
3,4,deer
4,5,frog
...,...,...
299995,299996,dog
299996,299997,automobile
299997,299998,cat
299998,299999,dog


In [16]:
resultDF[['id', 'label']].to_csv('kaggle_new.csv', index=False)

# fixing the submission

In [28]:
A = list(range(1, 300001))
for i in range(len(A)):
    A[i] = str(A[i])

sorting_permutation = np.argsort(A)
inverse_sorting_permutation = np.argsort(sorting_permutation)

In [32]:
data = pd.read_csv('kaggle_new.csv')

In [33]:
data = data.reindex(inverse_sorting_permutation)


In [34]:
data

,id,label
0,1,truck
111111,111112,airplane
222222,222223,automobile
233334,233335,ship
244445,244446,airplane
...,...,...
222218,222219,ship
222219,222220,deer
222220,222221,frog
222221,222222,frog


In [35]:
data['id'] = list(range(1, 300001))

In [36]:
data

,id,label
0,1,truck
111111,2,airplane
222222,3,automobile
233334,4,ship
244445,5,airplane
...,...,...
222218,299996,ship
222219,299997,deer
222220,299998,frog
222221,299999,frog


In [37]:
data.to_csv('kaggle_OK.csv', index=False)